In [47]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

from IPython.display import Video

%matplotlib inline

flow_params = {
    'pyr_scale': 0.5, 
    'levels': 5, 
    'winsize': 11, 
    'iterations': 5, 
    'poly_n': 5, 
    'poly_sigma': 1.1, 
    'flags': 0
}              

In [53]:
video = cv2.VideoCapture('road.mp4')
_, first_frame = video.read()
scale = .7
first_frame =  cv2.resize(first_frame, None, fx=scale, fy=scale)
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

In [54]:
h, w = prev_gray.shape
out = cv2.VideoWriter(
    'flow_road.mp4',
    cv2.VideoWriter_fourcc(*'DIVX'),
    video.get(cv2.CAP_PROP_FPS),
    (w, h)
)



In [55]:
mask = np.zeros_like(first_frame)
mask[:, :, 1] = 255

In [56]:
while True:
    _, frame = video.read()
    
    if frame is None:
        break
    
    #
    frame =  cv2.resize(frame, None, fx=scale, fy=scale)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    
    #
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, **flow_params)
    magnitude, angle = cv2.cartToPolar(flow[:, :, 0], flow[:, :, 1])
    
    
    #
    mask[:, :, 0] = angle/(2*np.pi)*180
    mask[:, :, 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    bgr_mask = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
    
    
    #
    flow_frame = cv2.addWeighted(frame, 1, bgr_mask, 2, 0) #=1*frame + 2*bgr_mask + 0
    out.write(flow_frame)
    
    prev_gray = gray

out.release()
video.release()

In [58]:
Video('flow_road.mp4')